# Custom features

Obtain features suitable for linking directly from contours.


In [ ]:
import colorcet  # noqa: F401  # Registers "cet_" prefixed colormaps in plt.colormaps
import janitor  # noqa: F401  # Registers methods on dataframes
import xarray as xr
from geopandas import GeoDataFrame, points_from_xy
from pandas import DataFrame, NamedAgg
from shapely import LinearRing
from src.boilercv.docs import HIDE

from boilercv.data import VIDEO, apply_to_img_da
from boilercv.data.sets import get_contours_df, get_dataset
from boilercv.docs import init, nowarn
from boilercv.images import scale_bool
from boilercv.images.cv import Op, Transform, transform
from boilercv.stages.experiments.e230920_subcool import GBC

init()


with nowarn(capture=True):
    import trackpy as tp


tp.quiet()

## Parameters

Given the 

In [ ]:
TIME = "2023-09-20T17:14:18"
"""Trial."""

FRAMES = [None, 1]
"""Frames.

A list that will become a slice. Not a tuple because `ploomber_engine` can't inject
tuples.
"""

HIDE

In [ ]:
PATH_TIME = TIME.replace(":", "-")
source_ds = get_dataset(PATH_TIME, stage="filled")
contours_df = get_contours_df(PATH_TIME)

## Find centers from filled contours using Trackpy

Synthesize bubble images by producing binary images with filled contours. Use Trackpy to find bubble centers.

In [ ]:
video = apply_to_img_da(
    lambda img: transform(img, Transform(Op.open, 12)),
    scale_bool(source_ds["video"]),
    vectorize=True,
)
GUESS_DIAMETER = 51  # (px) Guess diameter
objects = tp.batch(
    frames=video.sel(frame=slice(*FRAMES)).values,
    diameter=GUESS_DIAMETER,
    characterize=False,
).drop(columns="mass")
objects

,y,x,frame
0,437.1,208.9,0
1,436.6,223.0,1


## Find centers from contours using shapely

The prior approach throws out contour data, instead operating on filled contours. Instead, try using shapely to find centroids directly from contour data.

### Prepare to find objects

Prepare a dataframe with columns in a certain order, assign contour data to it and reset the hiearchical `frame` and `contour` indices to columns.

> **Note**
> `groupby` operations behave differently depending on whether there is a default index, a meaningful index, or a hierarchical index, so resetting the index. Additionally, `GBC` sets default values for `groupby`'s keyword arguments, enabling `observed` and `sort`, and disabling `as_index`, `dropna`, and `group_keys`.


In [ ]:
contours = (
    DataFrame(columns=["xpx", "ypx"])
    .assign(**contours_df)
    .rename(axis="columns", mapper=dict(xpx="x", ypx="y"))
    .reset_index()
    .assign(
        count=lambda df: df.groupby(["frame", "contour"], **GBC).x.transform("count")
    )
    .query("count > 3")
    .drop(columns="count")
    .assign(geometry=lambda df: points_from_xy(df.x, df.y))
    .groupby(["frame", "contour"], **GBC)
    .agg(
        x_mean=NamedAgg(column="x", aggfunc="mean"),
        y_mean=NamedAgg(column="y", aggfunc="mean"),
        centroid=NamedAgg(
            column="geometry", aggfunc=lambda df: LinearRing(df).centroid
        ),
    )
)
contours

,frame,contour,x_mean,y_mean,centroid
0,0,0,227.9,439.0,POINT (228.015 439.375)
1,0,1,203.6,437.4,POINT (202.637 437.271)
2,0,2,224.9,406.9,POINT (225.105 407.270)
3,0,3,180.0,298.8,POINT (180.189 298.593)
4,1,0,203.9,438.0,POINT (202.624 437.240)
5,1,1,227.0,438.3,POINT (228.150 439.035)
6,1,2,224.9,406.1,POINT (224.863 405.970)
7,1,3,180.0,298.0,POINT (180.000 298.000)
8,2,0,226.9,438.7,POINT (227.863 439.122)
9,2,1,203.0,435.8,POINT (202.551 437.025)


In [ ]:
my_objects = (
    GeoDataFrame(contours)
    .assign(x=lambda df: df.centroid.x, y=lambda df: df.centroid.y)
    .loc[:, ["x", "y", "frame"]]
)
my_objects

,x,y,frame
0,228.0,439.4,0
1,202.6,437.3,0
2,225.1,407.3,0
3,180.2,298.6,0
4,202.6,437.2,1
5,228.1,439.0,1
6,224.9,406.0,1
7,180.0,298.0,1
8,227.9,439.1,2
9,202.6,437.0,2


In [ ]:
contours = DataFrame(columns=["xpx", "ypx"]).assign(**contours_df)

ds = xr.zeros_like(source_ds, dtype=source_ds[VIDEO].dtype)
# video = ds[VIDEO]
# if not contours.empty:
#     for frame_num, frame in enumerate(video):
#         contours: list[ArrInt] = list(  # type: ignore  # pyright 1.1.333
#             contours.loc[frame_num, :]
#             .groupby("contour")
#             .apply(lambda grp: grp.values)  # type: ignore  # pyright 1.1.333
#         )
#         video[frame_num, :, :] = draw_contours(
#             scale_bool(frame.values), contours
#         )
# ds[VIDEO] = pack(video)
# ds = ds.drop_vars(ROI)

In [ ]:
my_objects

,x,y,frame
0,228.0,439.4,0
1,202.6,437.3,0
2,225.1,407.3,0
3,180.2,298.6,0
4,202.6,437.2,1
5,228.1,439.0,1
6,224.9,406.0,1
7,180.0,298.0,1
8,227.9,439.1,2
9,202.6,437.0,2
